## 이상치 제거 
### 실행 위치 input/code/remove_misslabel.ipynb 
#### 본인에게 맞는 실행위치로 파일을 이동시켜주세요
- 최소한의 데이터만 수정할 수 있도록, 최대한 엄격하게 선정하였습니다. 
- 그 결과 눈 검수를 통해 결정했던 약 100개의 결측치에서 55개만을 대상으로 진행하였습니다. 

In [191]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')


import cv2

import numpy as np
import pandas as pd
from tqdm import tqdm

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO


# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.patches import Patch

import ast
import copy

In [192]:
dataset_path  = '../data'
anns_file_path = dataset_path + '/' + 'train_all_new.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)
cat_names = [cat['name'] for cat in categories]

print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)


Number of categories: 10
Number of annotations: 26931
Number of images: 3467


In [176]:
miss_dataset_path = '../data/misslabel.csv'
df = pd.read_csv(miss_dataset_path)    

coco = COCO(anns_file_path)

loading annotations into memory...
Done (t=5.50s)
creating index...
index created!


In [193]:
tmp_anns = copy.deepcopy(anns)

for idx, row in tqdm(df.iterrows(), total = df.shape[0]):
    for tmp_ann in tmp_anns:
        if not tmp_ann['id'] in ast.literal_eval(row['annotation_id']):
            continue
        if int(row['category']) == -1:
            anns.remove(tmp_ann)
        else:
            anns[anns.index(tmp_ann)]['category_id'] = int(row['category'])

100%|██████████| 55/55 [00:26<00:00,  2.07it/s]


In [194]:
df

,Unnamed: 0,index,file_name,image_id,annotation_id,category
0,0,0,batch_01_vt/1076.jpg,838,[3612],-1
1,1,2,batch_03/0938.jpg,3221,[26107],10
2,2,3,batch_01_vt/0052.jpg,41,[288],8
3,3,4,batch_01_vt/0022.jpg,16,[84],8
4,4,5,batch_01_vt/0754.jpg,602,[2750],3
5,5,6,batch_01_vt/1151.jpg,887,[3869],1
6,6,7,batch_01_vt/1155.jpg,890,[3895],2
7,7,8,batch_03/0860.jpg,3169,[25828],2
8,8,9,batch_02_vt/0672.jpg,1465,"[10481, 10483, 10485]",10
9,9,10,batch_02_vt/1820.jpg,2360,[21758],-1


## 확인

In [195]:
#category_id 가 잘 바뀌었는가 
#[8944, 8945, 8946, 8947, 8948, 8949, 8950]	
for ann in anns:
    if ann['id'] in [18422, 18423, 18424, 18425, 18434]:
        print(ann['category_id'])

1
1
1
1
1


In [196]:
#annotation 삭제가 되었는가? 
flag = False
for ann in anns:
    if ann['id'] in [21758]:
        flag = True

if not flag:
    print("good! :)")


good! :)


In [197]:
categories

[{'id': 1, 'name': 'General trash', 'supercategory': 'General trash'},
 {'id': 2, 'name': 'Paper', 'supercategory': 'Paper'},
 {'id': 3, 'name': 'Paper pack', 'supercategory': 'Paper pack'},
 {'id': 4, 'name': 'Metal', 'supercategory': 'Metal'},
 {'id': 5, 'name': 'Glass', 'supercategory': 'Glass'},
 {'id': 6, 'name': 'Plastic', 'supercategory': 'Plastic'},
 {'id': 7, 'name': 'Styrofoam', 'supercategory': 'Styrofoam'},
 {'id': 8, 'name': 'Plastic bag', 'supercategory': 'Plastic bag'},
 {'id': 9, 'name': 'Battery', 'supercategory': 'Battery'},
 {'id': 10, 'name': 'Clothing', 'supercategory': 'Clothing'}]

In [198]:
file = dict()

In [199]:
file["images"] = imgs 
file["categories"] = categories
file["annotations"] = anns 

In [201]:
with open(f'../data/train_all_removed.json', 'w') as make_file:
    json.dump(file, make_file, indent='\t')